# Ibuprofen in gas phase

**Disclaimer: this tutorial does not work on Windows because the [openforcefield](https://openforcefield.org/) package is only released for MacOS and Linux. For now, it is recommended to run this notebook on a (Linux) HPC as explained in `03_elsewhere/01_run_openmm_on_a_hpc.ipynb`. Alternatively, Windows users may install [WSL](https://docs.microsoft.com/en-us/windows/wsl/install-win10) or [VirtualBox](https://www.virtualbox.org/) to run a Linux operating system locally, on which OpenMM and openforcefield can be installed.**

This notebook demonstrates a simple use case of the [Open Force Field (OFF)](https://openforcefield.org/) toolit. [OFF](https://openforcefield.org/) is a relatively new initiative and aims to bring more transparency into the process of generating force fields (for small molecules) through open science and open source principles.

We will create a force field for a small molecule, for which no standard force fields are available, using the [SMIRNOFF99Frosst](https://github.com/openforcefield/smirnoff99Frosst) set of parameters. The Smirnoff parameters cover a very broad set of organic small drug-like molecules. This parameterization is not yet intended to be competitive in accuracy with older initiatives like GAFF, OPLS or CGENFF. Instead it focuses more on extensibility, to enable incremental improvements, and exploitation of modern cheminformatics libraries. More details can be found here: [10.1021/acs.jctc.8b00640](https://pubs.acs.org/doi/10.1021/acs.jctc.8b00640)

This notebook uses ibuprofen as a simple test case. In principle any small drug-like molecule could be used instead and your are encouraged to try your favorite compound from [PubChem](https://pubchem.ncbi.nlm.nih.gov/).

## 1. Downloading and converting a molecule from PubChem

In [ ]:
# Download ibuprofen SDF file from PubChem.
from openbabel import openbabel
import requests

cid = '3672'  # This is the pubchem ID for ibuprofen
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/CID/{cid}/record/SDF/?record_type=3d&response_type=save"
with open(f"CID_{cid}.sdf", "w") as f:
    f.write(requests.get(url).text)

# Convert the SDF to a PDB file with openbabel
obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("sdf", "pdb")
mol = openbabel.OBMol()
# Open Babel will uncompress automatically
obConversion.ReadFile(mol, f"CID_{cid}.sdf")
obConversion.WriteFile(mol, f"CID_{cid}.pdb")

## 2. Assigning SMIRNOFF99Frosst parameters

Only in the following code, the Open Force Field toolkit is used.

In [ ]:
from openforcefield.topology import Molecule
from openforcefield.typing.engines.smirnoff import ForceField

# - Load the topology from the SDF file.
molecule = Molecule(f"CID_{cid}.sdf")

# - Load the SMIRNOFF99Frosst force field definition.
force_field = ForceField("test_forcefields/smirnoff99Frosst.offxml")

# - Create an OpenMM system object.
# There is a lot (!!) going on under the hood, including
# the calculation of atomic charges.
system = force_field.create_openmm_system(molecule.to_topology())

## 3. Performing a short molecular dynamics simulation

The code below contains nothing new compared to the molecular dynamics simulations on alanine dipeptide in section 2.

In [ ]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import numpy as np
from sys import stdout

pdb = PDBFile(f"CID_{cid}.pdb")
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 2*femtoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()

with open("init.pdb", "w") as f:
    PDBFile.writeFile(simulation.topology, pdb.positions, f)
simulation.reporters = []
simulation.reporters.append(DCDReporter('traj.dcd', 100))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
                                              temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("scalars.csv", 100, time=True,
                                              potentialEnergy=True, totalEnergy=True, temperature=True))
simulation.step(10000)

In [ ]:
import mdtraj
import nglview

traj = mdtraj.load('traj.dcd', top='init.pdb')
traj.superpose(traj, 0)
nglview.show_mdtraj(traj)